### 1. Implement a Fibonacci series generator.

In [15]:
def fibonacci(n):
    a,b = 0, 1
    
    while True:
        c= a + b
        if c >= n:
            break
        
        yield(c)
        a, b = b, c
    return

series = fibonacci(10)
print(series)
print([x for x in series])

<generator object fibonacci at 0x00000291D360E7B0>
[1, 2, 3, 5, 8]


### 2. Build a to-do list in python and use Postgres for persistent storage.

In [ ]:
import psycopg2

class Storage:
    # DB connection config
    DB_CONFIG = {
        'dbname': 'test',
        'user': 'postgres',
        'password': 'xxx',
        'host': 'localhost',
        'port': '5432'
    }

    def __init__(cls):
        pass

    def get_connection(cls): return psycopg2.connect(**cls.DB_CONFIG)

    def add_task(cls, task):
        with cls.get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute("INSERT INTO todos (task) VALUES (%s)", (task,))
            conn.commit()
            
        print("Task added.")

    def complete_task(cls, task_id):
        with cls.get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute("UPDATE todos SET completed = TRUE WHERE id = %s", (task_id,))
            conn.commit()
        print("Task marked as completed.")

    # Delete task
    def delete_task(cls, task_id):
        with cls.get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute("DELETE FROM todos WHERE id = %s", (task_id,))
            conn.commit()
        print("Task deleted.")


storage = Storage()

tasks_str = input("Enter your list of tasks sepearted by comma(,): ")
task_list = tasks_str.split(',')
print(f"storing a total of {len(task_list)} tasks.")
for i in task_list:
    storage.add_task(i.strip())

storing a total of 3 tasks.
Task added.
Task added.
Task added.


### 3. Create a Postgres database with tables and perform crud operations.

In [12]:
def create():
    with storage.get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS persons (
                    id SERIAL,
                    first_name TEXT,
                    last_name TEXT,
                    age INT,
                    PRIMARY KEY (id)
                );
            """)
        conn.commit()
    print('Table created')

def read():
    with storage.get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM persons")
            rows = cur.fetchall()
            if rows:
                for row in rows:
                    print(row)
            else:
                print(">>No data found<<")

def update():
    with storage.get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO persons (
                    first_name,
                    last_name,
                    age
                ) VALUES 
                    ('Ola', 'Hensen', 25),
                    ('Tove', 'Svendson', 23),
                    ('Kari', 'Petterson', 20);
            """)
        conn.commit()
    print("Table updated with 3 rows.")

def delete(id):
    with storage.get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM persons WHERE id = %s", (id,))
        conn.commit()
    print(f"Row {id} deleted.")

def crud():
    create()
    print("Read all data from table")
    read()
    update()
    delete(1)

crud()

Table created
Read all data from table
>>No data found<<
Table updated with 3 rows.
Row 1 deleted.


### 4. save babynames (extracted previously with regex) to postgres table.

In [7]:
from Assignment_2.Extract_baby_names import extract_names

def create_table():
    with storage.get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS baby_names (
                    id SERIAL,
                    male_names TEXT,
                    female_names TEXT,
                    PRIMARY KEY (id)
                );
            """)
        conn.commit()
    print('Table created')

def add_names(male_names, female_names):
    with storage.get_connection() as conn:
        with conn.cursor() as cur:
            for n in range(len(male_names)): 
                cur.execute("INSERT INTO baby_names (male_names, female_names) VALUES (%s, %s)", (male_names[n], female_names[n]))
        conn.commit()
    print("Names added.")

def main():
    file_path = 'Assignment_2/baby2008.html'
    male_names, female_names = extract_names(file_path)
    create_table()
    add_names(male_names, female_names)

main()

Table created
Names added.
